In [1]:
import gzip
import pickle
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import os
import keras
import tensorflow as tf
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Dense, BatchNormalization, Activation, Add, Flatten, Dropout
print(f'Is GPU Avaliable: {tf.config.list_physical_devices("GPU")}')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
plt.style.use('ggplot')

DATA_PATH = '/root/Workspace/DataWarehouse/stMary_RRpo'

with gzip.open(f'{DATA_PATH}/21_230531_resamp_sliced125_filt_stmary_train_patients.pickle.gzip', 'rb') as f:
    dataset = pickle.load(f)

print(len(dataset), len(dataset[0][0]))

random.seed(42)
random.shuffle(dataset)

pleths = []
resps = []
for ppg, rr in dataset:
    pleths.append(ppg.astype(np.float64))
    resps.append(rr)

pleths = np.asarray(pleths)
resps = np.asarray(resps)
print(pleths.shape, resps.shape)

scaler = MinMaxScaler()
scaled_pleths = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in pleths])
print(scaled_pleths.shape, type(scaled_pleths[0][0][0]))

ratio_tr = 0.8
train_x, train_y = scaled_pleths[:int(len(scaled_pleths)*ratio_tr)], resps[:int(len(resps)*ratio_tr)]
val_x, val_y = scaled_pleths[int(len(scaled_pleths)*ratio_tr):], resps[int(len(resps)*ratio_tr):]
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)

2023-05-31 11:36:32.150119: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-31 11:36:32.196262: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 11:36:32.893891: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
5224 1800
(5224, 1800) (5224,)
(5224, 1800, 1) <class 'numpy.float64'>
(4179, 1800, 1) (4179,)
(1045, 1800, 1) (1045,)


In [35]:
class Time2Vec(keras.layers.Layer):
    def __init__(self, output_dim=None, **kwargs):
        super(Time2Vec, self).__init__(**kwargs)
        self.output_dim = output_dim
        
        
    def build(self, input_shape): # (sample, timestep, feature)
        self.w = self.add_weight(name='orgW',
                                 shape=(input_shape[1], input_shape[-1]),
                                 initializer='uniform',
                                 trainable=True)
        self.p = self.add_weight(name='orgP',
                                 shape=(input_shape[1], input_shape[-1]),
                                 initializer='uniform',
                                 trainable=True)
        # (sample, timestep, output_dim)
        self.W = self.add_weight(name='sinW',
                                 shape=(input_shape[-1], self.output_dim-1),
                                 initializer='uniform',
                                 trainable=True)
        self.P = self.add_weight(name='sinP',
                                 shape=(input_shape[1], self.output_dim-1),
                                 initializer='uniform',
                                 trainable=True)
        
        super(Time2Vec, self).build(input_shape)
    
    
    def call(self, x):
        # 시계열 x에 지나친 변동을 주지 않고 되도록 온전하게 보존하는 linear f
        original = self.w * x + self.p
        sin_trans = tf.sin(tf.tensordot(x, self.W, axes=1) + self.P)
        
        # (sample, timestep, output_dim)
        return tf.concat([sin_trans, original], -1) 


class RespDNN(Model):
    def __init__(self, *args, **kwargs):
        super(RespDNN, self).__init__(*args, **kwargs)
        self.t2v1 = Time2Vec(output_dim=64)
        self.conv1 = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')
        self.bn1 = BatchNormalization()

        self.maxpool1 = MaxPooling1D(strides=2, padding='same')
        self.subsamp1 = Conv1D(filters=64, kernel_size=2, strides=2, padding='same')

        self.t2v2 = Time2Vec(output_dim=65)
        self.conv2 = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')
        self.bn2 = BatchNormalization()

        self.linear = Dense(50, activation='relu')
        self.outputs = Dense(1)

    
    def call(self, inputs, training=None, mask=None):
        v1 = self.t2v1(inputs)   # 주파수 정보
        c1 = self.conv1(inputs) # 형태학적 정보
        c1 = self.bn1(c1)
        c1 = Activation('relu')(c1)
        x1 = v1 + c1

        x2 = self.maxpool1(x1)
        inputs2 = self.subsamp1(inputs)
        x2 = x2 + inputs2
        print(x2.shape)

        v2 = self.t2v2(x2)   # 주파수 정보
        c2 = self.conv2(x2) # 형태학적 정보
        c2 = self.bn2(c2)
        c2 = Activation('relu')(c2)
        x2 = v2 + c2
        print(x2.shape)

        x = self.linear(x2)
        return self.outputs(x)

In [36]:
EPOCHS = 100000
BATCH_SIZE = 256
LR = 0.001
kf = KFold(n_splits=5)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    # ModelCheckpoint('../models/230522-Resnet', monitor='val_loss', save_best_only=True)
]

model = RespDNN()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=keras.losses.MeanAbsoluteError(),
    metrics=keras.metrics.MeanAbsoluteError()
)

In [37]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(BATCH_SIZE)

In [39]:
with tf.device('/GPU:0'):
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        callbacks=callbacks,
        validation_data=val_dataset
    )

Epoch 1/100000
21/21 [==============================] - 2s 81ms/step - loss: 4.4850 - mean_absolute_error: 4.4850 - val_loss: 4.6051 - val_mean_absolute_error: 4.6051 - lr: 1.0000e-06
Epoch 2/100000
21/21 [==============================] - 2s 80ms/step - loss: 4.4850 - mean_absolute_error: 4.4850 - val_loss: 4.6052 - val_mean_absolute_error: 4.6052 - lr: 1.0000e-06
Epoch 3/100000
21/21 [==============================] - 2s 80ms/step - loss: 4.4850 - mean_absolute_error: 4.4850 - val_loss: 4.6052 - val_mean_absolute_error: 4.6052 - lr: 1.0000e-06
Epoch 4/100000
21/21 [==============================] - 2s 80ms/step - loss: 4.4850 - mean_absolute_error: 4.4850 - val_loss: 4.6051 - val_mean_absolute_error: 4.6051 - lr: 1.0000e-06
Epoch 5/100000
21/21 [==============================] - 2s 80ms/step - loss: 4.4850 - mean_absolute_error: 4.4850 - val_loss: 4.6049 - val_mean_absolute_error: 4.6049 - lr: 1.0000e-06
Epoch 6/100000
21/21 [==============================] - 2s 80ms/step - loss: 4.4